<a href="https://colab.research.google.com/github/rafia-10/llm-tuning/blob/main/fine_tunning_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git config --global user.name "rafia-10"
!git config --global user.email "rafiakedir22@gmail.com"

In [ ]:
!git clone  https://github.com/rafia-10/llm-tuning.git

In [ ]:
%cd llm-tuning

In [ ]:
!pip install trl
!pip install bitsandbytes

## Import all the necessary libraries

In [ ]:
import torch
from trl import SFTTrainer,SFTConfig
from peft import LoraConfig
from datasets import load_dataset
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline)

## Loading the mode

In [ ]:
llama_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path = "aboonaji/llama2finetune-v2",
                                                   quantization_config = BitsAndBytesConfig(load_in_4bit = True,
                                                                                            bnb_4bit_compute_dtype = getattr(torch, "float16"),
                                                                                            bnb_4bit_quant_type = "nf4"))
llama_model.config.use_cache = False
llama_model.config.pretraining_tp = 1

## Loading the tokenizer

In [ ]:
llama_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path  = "aboonaji/llama2finetune-v2", trust_remote_code = True)
llama_tokenizer.pad_token= llama_tokenizer.eos_token
llama_tokenizer.padding_side = 'right'

## Setting the Training Argument

In [ ]:
training_arguments = SFTConfig(output_dir='./results', per_device_train_batch_size=4, max_steps=100, dataset_text_field = "text")

## Creating the Supervised fine-tuning trainer

In [ ]:
llama_sft_trainer = SFTTrainer(model = llama_model,
                               args = training_arguments,
                               train_dataset =load_dataset(path="aboonaji/wiki_medical_terms_llam2_format", split="train"),
                               peft_config = LoraConfig(task_type="CAUSAL_LM",r =64, lora_alpha=16, lora_dropout=0.1)
                               )